# Fact-checking по Википедии
Задача разбивается на несколько частей:
1.   Поиск релевантной статьи по тексту и заголовку.
2.   Поиск релевантных предложений в статье.
3.   Классификация предложений: подтверждает, не подтверждает, недостаточно сведений. 
4.   Агрегация сведений по всем найденным релеватным предложениям.

Исходное соревнование: FEVER 2019

Оригинальная статья: https://www.aclweb.org/anthology/D19-6612/

In [1]:
!pip install allennlp==0.8.3

     |████████████████████████████████| 5.6MB 18.5MB/s 
     |████████████████████████████████| 317kB 56.3MB/s 
     |████████████████████████████████| 880kB 51.4MB/s 
     |████████████████████████████████| 133kB 58.5MB/s 
     |████████████████████████████████| 317kB 60.7MB/s 
     |████████████████████████████████| 5.3MB 49.4MB/s 
     |████████████████████████████████| 245kB 62.4MB/s 
     |████████████████████████████████| 133kB 60.4MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 266kB 58.1MB/s 
     |████████████████████████████████| 30.9MB 105kB/s 
     |████████████████████████████████| 3.5MB 50.9MB/s 
     |████████████████████████████████| 71kB 10.7MB/s 
     |████████████████████████████████| 6.9MB 50.1MB/s 
     |████████████████████████████████| 276kB 50.8MB/s 
     |████████████████████████████████| 153kB 60.1MB/s 
     |████████████████████████████████| 81kB 12.1MB/s

In [2]:
!pip install -U spacy

     |████████████████████████████████| 10.4MB 16.1MB/s 
     |████████████████████████████████| 286kB 61.8MB/s 
     |████████████████████████████████| 1.1MB 55.1MB/s 
     |████████████████████████████████| 9.8MB 52.6MB/s 
ERROR: allennlp 0.8.3 has requirement spacy<2.2,>=2.0, but you'll have spacy 2.3.4 which is incompatible.
  Found existing installation: preshed 2.0.1
    Uninstalling preshed-2.0.1:
      Successfully uninstalled preshed-2.0.1
  Found existing installation: blis 0.2.4
    Uninstalling blis-0.2.4:
      Successfully uninstalled blis-0.2.4
  Found existing installation: thinc 7.0.8
    Uninstalling thinc-7.0.8:
      Successfully uninstalled thinc-7.0.8
  Found existing installation: spacy 2.1.9
    Uninstalling spacy-2.1.9:
      Successfully uninstalled spacy-2.1.9


In [3]:
!pip install pymediawiki==0.6.1

In [4]:
!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo

Cloning into 'bert_repo'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 317.85 KiB | 638.00 KiB/s, done.
Resolving deltas: 100% (185/185), done.


In [5]:
!pip uninstall -y tensorflow
!pip install -U tensorflow==1.14.0

Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 109.2MB 96kB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 491kB 56.3MB/s 
     |████████████████████████████████| 3.2MB 54.4MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0


In [6]:
!pip install -U tensorflow-gpu==1.14.0

     |████████████████████████████████| 377.0MB 43kB/s 


In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [47]:
#зайти на http://www.mediafire.com/file/8g81mb1joym25x0/model.ckpt-6343.zip/file и сгенерировать новую ссылку
!wget http://download2283.mediafire.com/i1iojcs6mtsg/8g81mb1joym25x0/model.ckpt-6343.zip

--2020-11-27 11:23:48--  http://download2283.mediafire.com/i1iojcs6mtsg/8g81mb1joym25x0/model.ckpt-6343.zip
Resolving download2283.mediafire.com (download2283.mediafire.com)... 199.91.155.24
Connecting to download2283.mediafire.com (download2283.mediafire.com)|199.91.155.24|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1176952097 (1.1G) [application/zip]
Saving to: ‘model.ckpt-6343.zip’

model.ckpt-6343.zip 100%[===================>]   1.10G  13.9MB/s    in 93s     

2020-11-27 11:25:21 (12.1 MB/s) - ‘model.ckpt-6343.zip’ saved [1176952097/1176952097]



In [48]:
!unzip model.ckpt-6343.zip -d checkps/

Archive:  model.ckpt-6343.zip
  inflating: checkps/model.ckpt-6343.data-00000-of-00001  
  inflating: checkps/model.ckpt-6343.index  
  inflating: checkps/model.ckpt-6343.meta  


# Document Retrieval

In [10]:
import warnings
warnings.filterwarnings("ignore")

from allennlp.predictors.predictor import Predictor
import ast
from mediawiki import MediaWiki
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
import os
import pandas as pd
import pickle
from unidecode import unidecode

In [11]:
data_path = os.path.dirname(os.path.abspath(''))
wiki = MediaWiki()
w = {}
ps = PorterStemmer()
constituency_parser = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/elmo-constituency-parser-2018.03.14.tar.gz")
dependency_parser = Predictor.from_path('https://s3-us-west-2.amazonaws.com/allennlp/models/biaffine-dependency-parser-ptb-2018.08.23.tar.gz')
#ner_parser = Predictor.from_path('https://s3-us-west-2.amazonaws.com/allennlp/models/ner-model-2018.12.18.tar.gz')


100%|██████████| 73228310/73228310 [00:07<00:00, 10433549.02B/s]
Did not use initialization regex that was passed: .*weight_ih.*
Did not use initialization regex that was passed: .*bias_ih.*
Did not use initialization regex that was passed: .*weight_hh.*
Did not use initialization regex that was passed: .*bias_hh.*


In [12]:
def extract_noun_phrase_from_tree(node):
    if node['nodeType'] == 'NP':
        yield node['word']
    for child in node.get('children', []):
        yield from extract_noun_phrase_from_tree(child)


def extract_titles(search, urls, stem_claim=None, stem=True):
    assert not stem or stem_claim is not None
    
    for p in wiki.search(search, results=3):    # top-7,5,2 works worse
        title = p.split('(')[0]
        if title == 'Trollhunter':
            title = 'Trollhunters'
        
        in_claim = True    # results filtering
        if stem:
            stem_title = [unidecode(ps.stem(word)) for word in word_tokenize(title)]
            for word in stem_title:
                if word not in stem_claim:
                    in_claim = False
        else:
            for word in title:
                if word not in claim:
                    in_claim = False
        if in_claim:
            #urls.add(unidecode(p.replace(' ', '_')))
            urls.add(p)
    return urls


def find_documents(claim, stem=True, use_ner=False):
    if stem:
        stem_claim = [unidecode(ps.stem(word)) for word in word_tokenize(claim)]
    
    res = constituency_parser.predict(sentence=claim)
    urls = set()
    for noun_phrase in extract_noun_phrase_from_tree(res['hierplane_tree']['root']):
        urls = extract_titles(noun_phrase, urls, stem_claim, stem)
    
    word = dependency_parser.predict(sentence=claim)['hierplane_tree']['root']['word']
    prev = claim.split(word)[0]
    if prev != '':
        urls = extract_titles(prev, urls, stem_claim, stem)
    
    if use_ner:
        res = ner_parser.predict(sentence=claim)
        ner = ' '.join(np.array(res['words'])[np.array(res['tags']) != 'O'])
        if ner != '':
            urls = extract_titles(ner, urls, stem_claim, stem)
    
    return urls


def document_retrieval(task_type, claims=None):
    assert task_type in ['test', 'train', 'eval', 'demo'], "the 'task_type' parameter must take one of four values: 'test', 'train', 'eval' or 'demo'"
    assert task_type != 'demo' or claims is not None, "claims list should be provided in the 'demo' task type"
    
    if task_type != 'demo':
        if task_type == 'train':
            train = pd.read_csv(os.path.join(data_path, "FEVER_data/train.csv"))
        if task_type == 'eval':
            train = pd.read_csv(os.path.join(data_path, "FEVER_data/shared_task_dev.csv"))
        if task_type == 'test':
            train = pd.read_csv(os.path.join(data_path, "FEVER_data/shared_task_test.csv"))
        claims = train.claim.values
        verifiable = train.verifiable.values
    
    documents = {}
    length = len(claims)
    for i in range(length):
        if task_type != 'train' or verifiable[i] != 'VERIFIABLE':
            claim = claims[i]
            urls = find_documents(claim)
            documents[i] = tuple(urls)

    with open('documents_{}.pickle'.format(task_type), 'wb') as f:
        pickle.dump(documents, f)

In [13]:
claims = ['Alexander Pushkin was born in Paris']
document_retrieval('demo', claims)
with open('documents_demo.pickle', 'rb') as f:
    documents = pickle.load(f)
documents

Your label namespace was 'pos'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.


{0: ('Paris',
  'Alexander Pushkin',
  'Paris (disambiguation)',
  'Alexander Pushkin (diamond)',
  'Paris Paris')}

# Sentence Retrieval

In [14]:
import csv
import io
import logging
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TweetTokenizer
import os
import random
from random import randint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import torch
from tqdm import tqdm_notebook as tqdm
from unidecode import unidecode

In [15]:
data_path = os.path.dirname(os.path.abspath(''))
wiki_path = os.path.join(data_path, "FEVER_data/wiki_pages.csv")
glove_path = os.path.join(data_path, 'glove.840B.300d.txt')
fasttext_path = os.path.join(data_path, 'cc.en.300.vec')
infersent_path = os.path.join(data_path, "encoder/infersent%s.pkl")

tknzr = TweetTokenizer()
stop = stopwords.words('english')
w = {}
w2v_embs = {}
vectorizer = TfidfVectorizer(ngram_range=(1, 1), lowercase=True, max_df=0.85, binary=True)

In [16]:
def w2v(word):
    return w2v_embs.get(word, np.zeros(300))


def delete_stopwords(sentence):
    return ' '.join([x.lower() for x in sentence.split(' ') if x.lower() not in stop and x.lower() not in string.punctuation])


def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


def find_sentences_in_document(mode, data, claim, title, claim_emb=None, model=None, word_vectors=None, claim_entities=None, argsort=False):
    if data != []:
        sents = coref_in_start_sents(data, title)
        text = coref_in_start(title, sents)
    else:
        if title not in w:
            return [0]
        # text = w[title].replace('  ', '  .  .').split(' . ')
        text = w[title].replace('\n', '.').split('.')

    if mode == 1:
        tfidf = vectorizer.fit_transform([claim] + text)
        sims = cosine_similarity(tfidf[0], tfidf).flatten()[1:]

    if argsort:
        return sims.argsort()
    else:
        return sims


def find_sentences(urls, coref, claim, mode, top, model=None, word_vectors=None):
    result = {}
    if mode in [3, 4, 5]:
        claim_emb = np.mean([w2v_embs[word] for word in tknzr.tokenize(claim) 
                             if word in w2v_embs and word not in stop and word not in string.punctuation], axis=0)
    elif mode == 9:
        claim_emb = model.encode([claim], tokenize=True)
    else:
        claim_emb = None
    
    sims = np.array([])
    lens = [0]
    titles = []
    
    for title in urls:
        # title = unidecode(title.replace('(', '-LRB-').replace(')', '-RRB-').replace(':' , '-COLON-'))
        try:
            data = coref.get(title, [])
            similarities = find_sentences_in_document(mode, data, claim, title, claim_emb, model, word_vectors)
            sims = np.concatenate([sims, similarities])
            lens.append(len(similarities) + lens[-1])
            titles.append(title)
        except:
            continue
    
    lens = np.array(lens)
    for ind in sims.argsort()[-top:]:
        res = lens[lens - ind <= 0].argmax()
        result.setdefault(titles[res], dict())
        result[titles[res]][ind - lens[res]] = sims[ind]
    
    return result


def save_sr_results(claims, documents, mode, task_type, total=None, top=20, use_coreference=False, ann_type='Stanford'):
    if total is None:
        total = len(claims)
    
    if use_coreference:
        corefs = read_corefs(ann_type)
    coref = {}
    
    if w2v_embs == {}:
        if mode == 3 or mode == 5:
            load_glove(glove_path)
        if mode == 4:
            load_fasttext(fasttext_path)     
    
    word_vectors = None
    model = None
    if mode == 6:
        model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
        word_vectors = model.wv
    
    if mode == 3:
        model = load_infersent(infersent_path)
    
    saver = []
    for i in tqdm(range(total)):
        urls = []
        claim = claims[i]
        urls = documents[i]
        if use_coreference and i < len(corefs):
            coref = corefs[i]
        result = find_sentences(urls, coref, claim, mode, top, model, word_vectors)
        saver.append(result)
    
    with open('results_{}.pickle'.format(task_type), 'wb') as f:
        pickle.dump(saver, f)


def create_bert_file(task_type, claims, mode=1, total=None, top=20, 
                     use_coreference_bert=False, use_coreference_search=False, ann_type='Stanford'):
        
    with open('documents_{}.pickle'.format(task_type), 'rb') as f:
        documents = pickle.load(f)

    for el in documents:
        for doc in documents[el]:
            try:
                w[doc] = wiki.page(doc).content
            except:
                pass
    
    save_sr_results(claims, documents, mode, task_type, total, top, use_coreference_search, ann_type)
    
    with open('results_{}.pickle'.format(task_type), 'rb') as f:
        saver = pickle.load(f)
    
    if use_coreference_bert:
        corefs = read_corefs(ann_type)
        new_corefs = {}
        for i in range(len(corefs)):
            for title in corefs[i]:
                new_corefs[title] = corefs[i][title]
        del corefs
    
    with open('pred_{}.tsv'.format(task_type), "w", encoding="utf-8") as file:
        f = csv.writer(file, delimiter='\t')
        f.writerow(["text_a", "text_b", "title", "sent", "index"])
        for i in tqdm(range(len(saver))):
            new_result = {}
            result = saver[i]
            text_b = claims[i]
            
            for title in result:
                for sent in result[title]:
                    new_result[(title, sent)] = result[title][sent]
            
            for el in list(sorted(new_result.items(), key=lambda kv: -kv[1])):
                doc = el[0][0]
                if doc in w:
                    t = doc
                    sent = el[0][1]
                    text = w[doc].replace('\n', '.').split('.')
                    
                    if use_coreference_bert:
                        try:
                            data = new_corefs[doc]
                            sents = coref_in_start_sents(data, doc, ann_type)
                            text = coref_in_start(doc, sents)
                        except:
                            pass
                    
                    if text[sent] != '' and text[sent].strip()[0] == '.':
                        text[sent] = text[sent].strip()[1:]
                    text_a = '# ' + unidecode(t.replace('_', ' ')) + ' # ' + text[sent] + ' . '
                    f.writerow([text_a, text_b, doc, sent, i])

                
def sentence_retrieval(task_type, claims=None, mode=1, total=None, top=20, 
                       use_coreference_bert=False, use_coreference_search=False, ann_type='Stanford'):
    assert task_type in ['test', 'eval', 'demo'], "the 'task_type' parameter must take one of four values: 'test', 'eval' or 'demo'"
    assert task_type != 'demo' or claims is not None, "claims list should be provided in the 'demo' task type"
    
    if task_type != 'demo':
        if task_type == 'eval':
            train = pd.read_csv(os.path.join(data_path, "FEVER_data/shared_task_dev.csv"))
        if task_type == 'test':
            train = pd.read_csv(os.path.join(data_path, "FEVER_data/shared_task_test.csv"))
        claims = train.claim.values
    
    create_bert_file(task_type, claims, mode, total, top, use_coreference_bert, use_coreference_search, ann_type)

In [17]:
w = {}
task_type = 'demo'
claims = ['Alexander Pushkin was born in Paris']
sentence_retrieval(task_type, claims)
pd.read_csv('pred_{}.tsv'.format(task_type), sep='\t').head()

,text_a,text_b,title,sent,index
0,# Alexander Pushkin # Alexander Pushkin .,Alexander Pushkin was born in Paris,Alexander Pushkin,421,0
1,# Alexander Pushkin # Alexander Pushkin .,Alexander Pushkin was born in Paris,Alexander Pushkin,419,0
2,# Alexander Pushkin # Alexander Pushkin .,Alexander Pushkin was born in Paris,Alexander Pushkin,417,0
3,# Alexander Pushkin # Pushkin was born into Ru...,Alexander Pushkin was born in Paris,Alexander Pushkin,6,0
4,# Alexander Pushkin (diamond) # It was mined ...,Alexander Pushkin was born in Paris,Alexander Pushkin (diamond),2,0


# Natural Language Inference

In [18]:
import sys

if not 'bert_repo' in sys.path:
    sys.path += ['bert_repo']

import datetime
import json
import logging
import modeling
import optimization
import os
import pprint
import random
import run_classifier
import string
import sys
import tensorflow as tf
import tokenization

In [19]:
logging.getLogger().setLevel(logging.INFO)

BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
logging.info('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))

OUTPUT_DIR = './'


TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 3e-5
NUM_TRAIN_EPOCHS = 1.3
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 128
NUM_TPU_CORES = 8
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
ITERATIONS_PER_LOOP = 1000
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

INFO:root:***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****


In [22]:
class FeverProcessor(run_classifier.DataProcessor):
    """Processor for the FEVER data set"""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "eval.tsv")), #dev
            "dev_matched")

    def get_test_examples(self, data_dir, path_pred):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, path_pred)), "test")

    def get_labels(self):
        """See base class."""
        return ["NOT ENOUGH INFO", "SUPPORTS", "REFUTES"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0 or line == []:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = tokenization.convert_to_unicode(line[0])
            text_b = tokenization.convert_to_unicode(line[1])
            if set_type == "test":
                label = "NOT ENOUGH INFO"
            else:
                label = tokenization.convert_to_unicode(line[2])
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
        return examples

    
def bert_model(task_type, file_type=None):
    assert task_type in ['test', 'train', 'eval'], "the 'task_type' parameter must take one of four values: 'test', 'train' or 'eval'"
    assert task_type != 'test' or file_type in ['test', 'demo', 'eval'], 'you must select a file type ("demo", "eval" or "test") in the case of testing'
    
    logging.info('preprocessing')
    
    processor = FeverProcessor()
    label_list = processor.get_labels()
    
    tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
    
    tpu_cluster_resolver = None
    run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    log_step_count_steps=100,
    keep_checkpoint_max=1,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))
    
    #tf.logging.set_verbosity(tf.logging.INFO)
    
    if task_type == 'train':
        train_examples = processor.get_train_examples(OUTPUT_DIR)
    else:
        train_examples = ['pad']
    num_train_steps = int(
        len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    if task_type == 'train':
        INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
    else:
        for file in os.listdir('checkps'):
            if file.endswith(".meta"):
                INIT_CHECKPOINT = os.path.join('checkps', file[:-5])
                break
    
    model_fn = run_classifier.model_fn_builder(
        bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
        num_labels=len(label_list),
        init_checkpoint=INIT_CHECKPOINT,
        learning_rate=LEARNING_RATE,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        use_tpu=False,
        use_one_hot_embeddings=False)

    estimator = tf.contrib.tpu.TPUEstimator(
        use_tpu=False,
        model_fn=model_fn,
        config=run_config,
        train_batch_size=TRAIN_BATCH_SIZE,
        eval_batch_size=EVAL_BATCH_SIZE)
    
    logging.info('done')
    
    if task_type == 'train':
        train_features = run_classifier.convert_examples_to_features(
        train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
        print('***** Started training at {} *****'.format(datetime.datetime.now()))
        print('  Num examples = {}'.format(len(train_examples)))
        print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
        tf.logging.info("  Num steps = %d", num_train_steps)
        train_input_fn = run_classifier.input_fn_builder(
            features=train_features,
            seq_length=MAX_SEQ_LENGTH,
            is_training=True,
            drop_remainder=True)  #False
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
        print('***** Finished training at {} *****'.format(datetime.datetime.now()))
        
    if task_type == 'eval':
        eval_examples = processor.get_dev_examples(OUTPUT_DIR)
        eval_features = run_classifier.convert_examples_to_features(
            eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
        print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
        print('  Num examples = {}'.format(len(eval_examples)))
        print('  Batch size = {}'.format(EVAL_BATCH_SIZE))
        # Eval will be slightly WRONG on the TPU because it will truncate
        # the last batch.
        eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
        eval_input_fn = run_classifier.input_fn_builder(
            features=eval_features,
            seq_length=MAX_SEQ_LENGTH,
            is_training=False,
            drop_remainder=False)
        result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
        print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
        print("***** Eval results *****")
        for key in sorted(result.keys()):
            print('  {} = {}'.format(key, str(result[key])))

    if task_type == 'test':
        PREDICT_BATCH_SIZE = 5
        estimator = tf.contrib.tpu.TPUEstimator(
            use_tpu=False,
            model_fn=model_fn,
            config=run_config,
            train_batch_size=TRAIN_BATCH_SIZE,
            eval_batch_size=EVAL_BATCH_SIZE,
            predict_batch_size=PREDICT_BATCH_SIZE
        )
        
        predict_examples = processor.get_test_examples(OUTPUT_DIR, 'pred_{}.tsv'.format(file_type))
            
        num_actual_predict_examples = len(predict_examples)

        predict_file = os.path.join(OUTPUT_DIR, "predict.tf_record")
        run_classifier.file_based_convert_examples_to_features(predict_examples, label_list,
                                                MAX_SEQ_LENGTH, tokenizer,
                                                predict_file)

        tf.logging.info("***** Running prediction*****")
        tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                        len(predict_examples), num_actual_predict_examples,
                        len(predict_examples) - num_actual_predict_examples)
        tf.logging.info("  Batch size = %d", PREDICT_BATCH_SIZE)

        predict_drop_remainder = True
        predict_input_fn = run_classifier.file_based_input_fn_builder(
            input_file=predict_file,
            seq_length=MAX_SEQ_LENGTH,
            is_training=False,
            drop_remainder=predict_drop_remainder)

        result = estimator.predict(input_fn=predict_input_fn)

        output_predict_file = os.path.join(OUTPUT_DIR, "results_{}.csv".format(file_type))

        with tf.gfile.GFile(output_predict_file, "w") as writer:
            num_written_lines = 0
            tf.logging.info("***** Predict results *****")
            for (i, prediction) in enumerate(result):
                probabilities = prediction["probabilities"]
                if i >= num_actual_predict_examples:
                    break
                output_line = ",".join(
                    str(class_probability)
                    for class_probability in probabilities) + "\n"
                writer.write(output_line)
                num_written_lines += 1


In [49]:
bert_model('test', 'demo')

INFO:root:preprocessing


INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f65ec6ff710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, ini

INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f65ec6ff710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, ini

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


...

INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:prediction_loop marked as finished


INFO:tensorflow:prediction_loop marked as finished


INFO:tensorflow:prediction_loop marked as finished


INFO:tensorflow:prediction_loop marked as finished


In [50]:
bert_res = pd.read_csv('results_demo.csv', header=None)
bert_res.head()

,0,1,2
0,0.998976,0.000523,0.000501
1,0.998976,0.000523,0.000501
2,0.998976,0.000523,0.000501
3,0.302037,0.004985,0.692977
4,0.994911,0.000933,0.004157


# Aggregation

In [51]:
import ast
import json
import logging
import numpy as np
import pandas as pd
import pickle
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm import tqdm_notebook as tqdm
from unidecode import unidecode

In [52]:
data_path = os.path.dirname(os.path.abspath(''))
answers = ['SUPPORTS', 'REFUTES', 'NOT ENOUGH INFO']    # used to replace numbers and corresponding values

In [53]:
def get_prediction_format(claims, pred, bert_res, k=20):
    logging.info('prepare data')
    comb = pred.join(bert_res)
    comb['SUPPORTS'] = comb['SUPPORTS'].fillna(0.0)
    comb['REFUTES'] = comb['REFUTES'].fillna(0.0)
    comb['NOT ENOUGH INFO'] = comb['NOT ENOUGH INFO'].fillna(1.0)

    predictions = []
    for i in range(len(claims)):
        prediction = list(comb.loc[comb['index'] == i][answers].values.flatten())
        while len(prediction) < k * 3:
            prediction += [0.0, 0.0, 1.0]    # padding with NEI prediction
        predictions.append(prediction[:k * 3])
    predictions = np.array(predictions)
    logging.info('done')
    return predictions


def logical_aggregation(predictions, k=20):
    preds = []
    for prediction in predictions:
        scores = []
        ans = []
        for i in range(k):
            ans.append(answers[prediction[i * 3: (i + 1) * 3].argmax()])
            scores.append(np.max(prediction[i * 3: (i + 1) * 3]))
        s = ans.count('SUPPORTS')
        r = ans.count('REFUTES')
        if s == 0 and r == 0:
            preds.append('NOT ENOUGH INFO')
        elif s == 0:
            preds.append('REFUTES')
        elif r == 0:
            preds.append('SUPPORTS')
        else:
            l = ''
            cur_max = 0
            for i in range(len(ans)):
                if ans[i] != 'NOT ENOUGH INFO' and abs(scores[i]) > cur_max:
                    l = ans[i]
                    cur_max = abs(scores[i])
            preds.append(l)
    return np.array(preds)


def get_sents(label, bert, pred):
    if label == 'NOT ENOUGH INFO':
        return []
    sents = set()
    #bert = bert.sort_values(by=['Relevance'], ascending=False)
    for i, r in bert.iterrows():
        res = np.array([r['SUPPORTS'], r['REFUTES'], r['NOT ENOUGH INFO']])
        title = pred.loc[i]['title']
        sent = pred.loc[i]['sent']
        text_sent = pred.loc[i]['text_a']
        if answers[np.argmax(res)] == label and len(sents) < 5:
            sents.add(tuple([title, sent, text_sent, answers[np.argmax(res)]]))
    return list(sents)


def get_sents_with_labels(preds, results, bert_res, pred):
    total = len(results)
    predict_sents = []
    predict_labels = []
    start = 0
    for i in tqdm(range(total)):
        end = start
        result = results[i]
        label = preds[i]
        if result == {}:
            predict_labels.append(answers[2])
            predict_sents.append([])
        else:
            while end < len(pred) and pred.iloc[end]['index'] == pred.iloc[start]['index']:    # search of the relevant part
                end += 1
            bert = bert_res.iloc[start: end - 1]
            start = end
            sents = get_sents(label, bert, pred)
            predict_labels.append(label)
            predict_sents.append(sents)
    return predict_labels, predict_sents


def fever_output_format(predict_labels, predict_sents, ids, out_path='predictions.jsonl'):
    total = len(predict_labels)
    with open(out_path, 'w', encoding='utf-8') as outfile:  
        for j in tqdm(range(total)):
            id = int(ids[j])
            predicted_label = predict_labels[j]
            pred_evidence = predict_sents[j]
            predicted_evidence = []
            
            if predicted_label == 'NOT ENOUGH INFO':
                title = None
                sent = None
            else:
                for pred_p in pred_evidence:
                    if pred_p[0] in w:
                        title = w[pred_p[0]]
                        title = pred_p[0]
                        sent = int(pred_p[1])
                        predicted_evidence.append([title, sent])
            
            data = {
                    "id": id,
                    "predicted_label": predicted_label,
                    "predicted_evidence": predicted_evidence
                }
            json.dump(data, outfile, ensure_ascii=False)
            outfile.write('\n')

            
def aggregation(task_type, claims=None):
    assert task_type in ['test', 'demo'], "the 'task_type' parameter must take one of two values: 'test' or 'demo'"
    assert task_type == 'test' or claims is not None, "you must provide claims list for 'demo' task type"
    
    bert_res = pd.read_csv('results_{}.csv'.format(task_type), header=None).rename(columns={0:"NOT ENOUGH INFO", 1:"SUPPORTS", 2:"REFUTES"})
    pred = pd.read_csv('pred_{}.tsv'.format(task_type), sep='\t')
    
    #rank = pd.read_csv('results/{}_ranking.csv'.format(task_type), header=None)[[1]].rename(columns={1:'Relevance'})
    
    with open('results_{}.pickle'.format(task_type), 'rb') as f:
        results = pickle.load(f)
    
    if task_type == 'test':
        train = pd.read_csv(data_path + "/FEVER_data/shared_task_test.csv") 
        claims = train.claim.values
        ids = train.id.values
    else:
        ids = np.arange(len(claims))
    
    predictions = get_prediction_format(claims, pred, bert_res)
    preds = logical_aggregation(predictions)
    
    #bert_res = bert_res.join(rank)
    predict_labels, predict_sents = get_sents_with_labels(preds, results, bert_res, pred)
    fever_output_format(predict_labels, predict_sents, ids)
    return predictions

In [54]:
predictions = aggregation('demo', claims)

INFO:root:prepare data
INFO:root:done


In [55]:
def results_output():
    with open('predictions.jsonl', 'r') as f:
        for line in f:
            res = json.loads(line)
            print('Claim:', claims[res['id']])
            print('Verdict:', res['predicted_label'])
            print('Top evidence:')
            for ev in res['predicted_evidence']:
                print('\t', ev[0], '->', w[ev[0]].replace('\n', '.').split('.')[ev[1]])
            if len(res['predicted_evidence']) == 0:
                print('\t ---')
            print('\n\n')

In [56]:
results_output()

Claim: Alexander Pushkin was born in Paris
Verdict: REFUTES
Top evidence:
	 Alexander Pushkin -> Born in Moscow, Pushkin was entrusted to nursemaids and French tutors, and spoke mostly French until the age of ten
	 Alexander Pushkin -> Pushkin was born into Russian nobility in Moscow





# Pipeline

In [57]:
def pipeline(claims,task_type='demo'):
    document_retrieval(task_type, claims)
    sentence_retrieval(task_type, claims)
    bert_model('test', task_type)
    aggregation(task_type, claims)

In [58]:
claims = ['Vladimir Putin is a Russian swimmer', 'Vladimir Putin is the Russian president', 'Vladimir Putin is the American president']
w = {}
pipeline(claims)

INFO:root:preprocessing


INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f65a72e1cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, ini

INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f65a72e1cf8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, ini

INFO:tensorflow:_TPUContext: eval_on_tpu True


INFO:tensorflow:_TPUContext: eval_on_tpu True


...

INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:prediction_loop marked as finished


INFO:tensorflow:prediction_loop marked as finished


INFO:tensorflow:prediction_loop marked as finished


INFO:tensorflow:prediction_loop marked as finished
INFO:root:prepare data
INFO:root:done


In [59]:
results_output()

Claim: Vladimir Putin is a Russian swimmer
Verdict: NOT ENOUGH INFO
Top evidence:
	 ---



Claim: Vladimir Putin is the Russian president
Verdict: SUPPORTS
Top evidence:
	 Vladimir Putin -> Vladimir Vladimirovich Putin (; Russian: Владимир Владимирович Путин [vɫɐˈdʲimʲɪr vɫɐˈdʲimʲɪrəvʲɪtɕ ˈputʲɪn] (listen); born 7 October 1952) is a Russian politician and a former officer of the KGB who has served as President of Russia since 2012, previously holding the position from 1999 until 2008
	 Vladimir Putin -> In April 2008, Putin became the first Russian President who visited Libya



Claim: Vladimir Putin is the American president
Verdict: REFUTES
Top evidence:
	 Vladimir Putin -> Vladimir Vladimirovich Putin (; Russian: Владимир Владимирович Путин [vɫɐˈdʲimʲɪr vɫɐˈdʲimʲɪrəvʲɪtɕ ˈputʲɪn] (listen); born 7 October 1952) is a Russian politician and a former officer of the KGB who has served as President of Russia since 2012, previously holding the position from 1999 until 2008



